# Binance Currency Pair Analysis



# 1. Introduction

The purpose of this analysis is to examine the trading activity of various currency pairs involving fiat currencies on Binance. We will start by retrieving the necessary data, filtering for relevant pairs, and then proceed to analyze the data, with a particular focus on trading volume in USD for key cryptocurrencies like BTC, ETH, and USDT. This analysis aims to provide insights into the trading patterns and market behavior on the Binance platform.

In this notebook, we will conduct a general evaluation of the data provided by the Binance API, focusing on currency pairs that involve fiat currencies. The goal is to identify and analyze the available pairs, as well as to detect any new fiat currencies that may have been added to the platform.

## 1.1 Loading Necessary Libraries
In this section, we will import the libraries required for making API requests, handling data, and managing files. These include:

In [1]:
import requests  # To handle HTTP requests to the Binance API
import pandas as pd  # To structure and manipulate the data obtained
import os  # To manage directories for saving the data

## 1.2 Setting Up Directories**
We will set up a base directory to store the data files downloaded from Binance. This ensures that all data is organized and easily accessible for further analysis.

In [2]:
# Definir la carpeta base donde se guardarán los archivos CSV
base_folder = 'datos_historicos_binance'

# Crear la carpeta base si no existe
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# 2. Data Retrieval and Download

## 2.1 Retrieving Information on Currency Pairs in Binance

In this section, we will retrieve information on all available currency pairs on Binance using their API. Our goal is to identify pairs that involve fiat currencies, as these will be the focus of our analysis.



In [3]:
def obtener_info_pares_binance():
    # URL del endpoint de exchangeInfo de Binance
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    # Realizar la solicitud GET al endpoint
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir los datos de respuesta a formato JSON
        data = response.json()
        
        # Crear un diccionario para almacenar la información sobre los pares de divisas
        info_pares = {}
        
        # Iterar sobre los datos para obtener la información relevante sobre cada par de divisas
        for symbol_info in data['symbols']:
            symbol = symbol_info['symbol']
            base_currency = symbol_info['baseAsset']
            quote_currency = symbol_info['quoteAsset']
            
            # Almacenar la información en el diccionario
            info_pares[symbol] = {'base_currency': base_currency, 'quote_currency': quote_currency}
        
        return info_pares
    else:
        print(f"Error al obtener información sobre los pares de divisas de Binance. Código de estado: {response.status_code}")
        return None

# Obtener información sobre los pares de divisas
info_pares = obtener_info_pares_binance()

## 2.2 Filtering Currency Pairs by Fiat Currencies

Once we have retrieved the currency pair information, we will filter the pairs to focus only on those that involve fiat currencies. This step is essential for narrowing down our analysis to the most relevant pairs.


In [4]:
# Lista completa de monedas fiat
fiat_currencies = [
    'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'RUB', 
    'BRL', 'ARS', 'NGN', 'TRY', 'MXN', 'ZAR', 'PLN', 'SEK', 'NOK', 'DKK', 
    'HKD', 'SGD', 'NZD', 'KRW', 'THB', 'MYR', 'IDR', 'PHP', 'VND', 'CZK', 
    'HUF', 'RON', 'BGN', 'HRK', 'RSD', 'UAH', 'KZT', 'AZN', 'GEL', 'BYN',
    'EGP', 'ILS', 'SAR', 'AED', 'QAR', 'KWD', 'BHD', 'OMR', 'TND', 'MAD', 
    'DZD', 'TWD', 'LKR', 'PKR', 'BDT', 'KES', 'TZS', 'UGX', 'GHS', 'XAF', 
    'XOF', 'XPF', 'MUR', 'SCR', 'MGA', 'ZMW', 'MWK', 'BWP', 'NAD', 'SLL',
    'LRD', 'LSL', 'SZL', 'MZN', 'BIF', 'CDF', 'DJF', 'ETB', 'GNF', 'HTG',
    'LRD', 'MGA', 'NPR', 'PGK', 'PGK', 'SHP', 'SLL', 'SOS', 'SSP', 'STD',
    'SYP', 'TOP', 'TTD', 'TWD', 'VUV', 'WST', 'XCD', 'YER', 'ZMW'
]

In [5]:
# Filtrar para obtener solo los pares de interés con las monedas fiat
fiat_pairs = [pair for pair, currencies in info_pares.items() if currencies['quote_currency'] in fiat_currencies or currencies['base_currency'] in fiat_currencies]

print(f"Total pairs with fiat currencies: {len(fiat_pairs)}")


Total pairs with fiat currencies: 454


## 2.3 Downloading and Saving Historical Data for Fiat-Crypto Pairs on Binance

In this section, we will download historical daily data for the fiat-crypto pairs we've identified from Binance's API. The data will be saved in monthly CSV files, organized by fiat currency and cryptocurrency.

In [6]:
import requests
import pandas as pd
import os
from tqdm.notebook import tqdm

# Define the base folder where CSV files will be stored
base_folder = 'datos_historicos_binance'

# Create the base folder if it does not exist
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# Initialize a counter and a list to track saved files
total_files_saved = 0
saved_files = []

def obtener_datos_y_guardar_csv(symbol, base_currency, quote_currency):
    global total_files_saved
    global saved_files

    # Build the URL for the Binance API
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d'
    
    # Send a GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response data to JSON format
        data = response.json()
        
        # Create a pandas DataFrame with the data
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 
            'close_time', 'quote_asset_volume', 'number_of_trades', 
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        # Convert the 'timestamp' column to datetime format
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Determine the fiat currency and crypto currency based on their position
        if base_currency in fiat_currencies:
            fiat_currency = base_currency
            crypto_currency = quote_currency
        else:
            fiat_currency = quote_currency
            crypto_currency = base_currency
        
        # Create a folder for the fiat currency if it doesn't exist
        fiat_folder_path = os.path.join(base_folder, fiat_currency)
        if not os.path.exists(fiat_folder_path):
            os.makedirs(fiat_folder_path)
        
        # Create a subfolder for the cryptocurrency if it doesn't exist
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if not os.path.exists(crypto_folder_path):
            os.makedirs(crypto_folder_path)
        
        # Save the data into monthly CSV files with daily data inside
        for month, df_month in df.groupby(df['timestamp'].dt.to_period('M')):
            csv_file_path = os.path.join(crypto_folder_path, f'{fiat_currency.lower()}_{crypto_currency.lower()}_{month.start_time.strftime("%Y-%m")}.csv')
            df_month.to_csv(csv_file_path, index=False)
            total_files_saved += 1
            saved_files.append(csv_file_path)
    else:
        print(f"Error fetching data for {symbol}. Status code: {response.status_code}")

# Download data for all the pairs of interest with a progress bar
for pair in tqdm(fiat_pairs, desc="Processing pairs"):
    base_currency = info_pares[pair]['base_currency']
    quote_currency = info_pares[pair]['quote_currency']
    obtener_datos_y_guardar_csv(pair, base_currency, quote_currency)

# Summary message
print(f"Total files saved: {total_files_saved}")

Processing pairs:   0%|          | 0/454 [00:00<?, ?it/s]

Total files saved: 5795


## 2.3 Loading and Combining the Data for Analysis

In this section, we load the previously saved CSV files, which contain the historical data for each fiat-crypto pair. We then combine these data files into a single DataFrame to facilitate further analysis. This step is crucial for ensuring that our data is well-organized and ready for more detailed exploration.


In [7]:
import os
import pandas as pd
from tqdm.notebook import tqdm

# Initialize a list to store the DataFrames
dfs = []

# Get the list of fiat currencies to iterate over and initialize the progress bar
fiat_currencies = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]

# Iterate through the fiat currency folders with a progress bar
for fiat_currency in tqdm(fiat_currencies, desc="Processing fiat currencies"):
    fiat_folder_path = os.path.join(base_folder, fiat_currency)
    
    # Iterate through the crypto currency folders
    for crypto_currency in os.listdir(fiat_folder_path):
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if os.path.isdir(crypto_folder_path):
            # Iterate through the CSV files
            for csv_file in os.listdir(crypto_folder_path):
                csv_file_path = os.path.join(crypto_folder_path, csv_file)
                if csv_file_path.endswith('.csv'):
                    # Create a pandas DataFrame from the CSV data
                    df = pd.read_csv(csv_file_path)
                    # Add columns for fiat and crypto currencies
                    df['fiat_currency'] = fiat_currency
                    df['crypto_currency'] = crypto_currency
                    # Append the DataFrame to the list
                    dfs.append(df)

# Combine all the DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

Processing fiat currencies:   0%|          | 0/15 [00:00<?, ?it/s]

Check if there are any missing values in the dataset

In [8]:
# Check if there are any missing values in the dataset
if final_df.isnull().sum().sum() == 0:
    print("Data check complete: No missing values. All data is clean and ready for analysis.")
else:
    print("Data check complete: There are missing values in the dataset. Further inspection is needed.")

# Basic descriptive statistics (can be optional or removed if not needed for output)
final_stats = final_df.describe()
print("Basic statistics generated. Data is ready for further analysis.")

Data check complete: No missing values. All data is clean and ready for analysis.
Basic statistics generated. Data is ready for further analysis.


# 3. Data Analysis

## 3.1 Summary of Cryptocurrency Pairs Distribution Across Fiat Currencies

In this section, we explore the distribution of fiat currencies and their pairing with major cryptocurrencies like BTC and USDT. By examining the fiat currencies present in the dataset, particularly focusing on the last month's data, we can identify which fiat currencies are commonly paired with these cryptocurrencies. Additionally, we count the number of unique cryptocurrencies associated with each fiat currency to understand the diversity of crypto-fiat pairs across different fiat currencies.

In [9]:
import pandas as pd

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Get the list of all fiat currencies in the dataset
all_fiat_currencies = final_df['fiat_currency'].unique()

# Get the list of fiat currencies present in the last month's data
fiat_currencies_last_month = filtered_df['fiat_currency'].unique()

# Identify fiat currencies that have BTC pairs
btc_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'BTC']['fiat_currency'].unique()

# Identify fiat currencies that have USDT pairs
usdt_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'USDT']['fiat_currency'].unique()

# Identify fiat currencies without BTC pairs
fiat_without_btc = set(fiat_currencies_last_month) - set(btc_fiat_currencies)

# Identify fiat currencies without USDT pairs
fiat_without_usdt = set(fiat_currencies_last_month) - set(usdt_fiat_currencies)

# Identify fiat currencies without either BTC or USDT pairs
fiat_without_btc_or_usdt = fiat_without_btc.union(fiat_without_usdt)

# Counting the number of unique cryptos per fiat currency
crypto_count_per_fiat = final_df.groupby('fiat_currency')['crypto_currency'].nunique()

# Combine all this information into a summary table
summary_table = pd.DataFrame({
    "All Fiat Currencies": [sorted(all_fiat_currencies)],
    "Fiat Currencies in Last Month": [sorted(fiat_currencies_last_month)],
    "Fiat Currencies with BTC Pairs": [sorted(btc_fiat_currencies)],
    "Fiat Currencies with USDT Pairs": [sorted(usdt_fiat_currencies)],
    "Fiat Currencies without BTC Pairs": [sorted(fiat_without_btc)],
    "Fiat Currencies without USDT Pairs": [sorted(fiat_without_usdt)],
    "Fiat Currencies without BTC or USDT Pairs": [sorted(fiat_without_btc_or_usdt)]
})

# Display the summary table as text to make it more readable
for column in summary_table.columns:
    print(f"\n{column} (Total {len(summary_table[column][0])}):")
    print(", ".join(summary_table[column][0]))

# Add a sub-sub-title for the next section
print("\n### Number of Unique Cryptocurrencies per Fiat Currency ###")

# Display the count of unique cryptos per fiat currency in a more readable format
for fiat, count in crypto_count_per_fiat.items():  # Corrected to use items() instead of iteritems()
    print(f"{fiat}: {count} unique cryptocurrencies")



All Fiat Currencies (Total 15):
ARS, AUD, BRL, CZK, EUR, GBP, JPY, MXN, NGN, PLN, RON, RUB, TRY, UAH, ZAR

Fiat Currencies in Last Month (Total 11):
ARS, BRL, CZK, EUR, JPY, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies with BTC Pairs (Total 10):
ARS, BRL, EUR, JPY, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies with USDT Pairs (Total 10):
ARS, BRL, CZK, EUR, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies without BTC Pairs (Total 1):
CZK

Fiat Currencies without USDT Pairs (Total 1):
JPY

Fiat Currencies without BTC or USDT Pairs (Total 2):
CZK, JPY

### Number of Unique Cryptocurrencies per Fiat Currency ###
ARS: 3 unique cryptocurrencies
AUD: 30 unique cryptocurrencies
BRL: 47 unique cryptocurrencies
CZK: 1 unique cryptocurrencies
EUR: 64 unique cryptocurrencies
GBP: 25 unique cryptocurrencies
JPY: 7 unique cryptocurrencies
MXN: 3 unique cryptocurrencies
NGN: 10 unique cryptocurrencies
PLN: 4 unique cryptocurrencies
RON: 4 unique cryptocurrencies
RUB: 25 unique cryptocurrencies
TRY: 2

In [50]:
## Visualizing the Unique Frequency of Cryptocurrencies

# In this section, we focus on visualizing the frequency of unique crypto-fiat pairs in the last month's data. By filtering the dataset and eliminating duplicates, we identify how frequently each cryptocurrency is paired with different fiat currencies. The resulting bar chart provides a visual representation of the popularity of each cryptocurrency in the most recent trading period.

import pandas as p
import plotly.graph_objects as go

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Drop duplicates to count each crypto-fiat pair only once
unique_pairs_df = filtered_df.drop_duplicates(subset=['crypto_currency', 'fiat_currency'])

# Count the frequency of each cryptocurrency in the filtered dataset
crypto_frequency = unique_pairs_df['crypto_currency'].value_counts()

# Create the Plotly bar chart
fig1 = go.Figure(data=[
    go.Bar(x=crypto_frequency.index, y=crypto_frequency.values, marker_color='skyblue')
])

# Customize the layout
fig1.update_layout(
    title="Unique Frequency of Cryptocurrencies in the Last Month",
    xaxis_title="Cryptocurrency",
    yaxis_title="Unique Frequency",
    xaxis_tickangle=-45,
)

# Show the plot
fig1.show()

# Simple confirmation message
print("Unique frequency plot of cryptocurrencies for the last month generated.")


Unique frequency plot of cryptocurrencies for the last month generated.


In [12]:
## Heatmap of Top 10 Cryptocurrencies Paired with Fiat Currencies

# This section presents a heatmap that visualizes the presence of trading pairs between the top 10 cryptocurrencies and various fiat currencies. The heatmap offers a clear overview of which fiat currencies are most commonly paired with these leading cryptocurrencies. Additionally, we include a category labeled 'Others' to capture any cryptocurrencies outside the top 10, ensuring a comprehensive view of the dataset.

import pandas as pd
import plotly.graph_objects as go

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = crypto_counts.nlargest(10).index.tolist()

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month].copy()  # Use .copy() to avoid the SettingWithCopyWarning

# Group the remaining cryptocurrencies into "Others"
filtered_df.loc[:, 'crypto_category'] = filtered_df['crypto_currency'].apply(
    lambda x: x if x in top_10_cryptos else 'Others'
)

# Create a list of unique categories (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# List of all fiat currencies present in the entire dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize an empty matrix with zeros (no pairs) for all fiat currencies
matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

# Fill the matrix with 1 where pairs exist
for _, row in filtered_df.iterrows():
    matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

# Unified color scheme for all cryptos
colorscale = [[0, '#D3D3D3'], [1, '#4682B4']]  # Light gray for no pair, steel blue for all pairs

# Plot using Plotly
fig = go.Figure(data=go.Heatmap(
    z=matrix.values,
    x=matrix.columns,
    y=matrix.index,
    colorscale=colorscale,
    showscale=False,  # Hide color scale bar
))

# Update layout for better readability and custom Y-axis labels
fig.update_layout(
    title="Top 10 Crypto-Fiat Pair Presence and 'Others'",
    xaxis_title="Fiat Currency",
    yaxis_title="Cryptocurrency",
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(len(categories))),
        ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
        tickfont=dict(size=10),
        side='left'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(all_fiats))),
        ticktext=all_fiats  # Ensure all fiat currencies are shown on the X-axis
    ),
    template="plotly_white"
)

# Show the plot
fig.show()


In [57]:
# Animated Heatmap of Crypto-Fiat Pairs Over Time

# In this section, we extend the heatmap analysis by introducing a time element, creating an animated heatmap that shows the presence of crypto-fiat pairs over a series of months. This visualization helps track changes over time, providing insights into the evolving dynamics of crypto-fiat pairings in the market. The animation allows us to see how the presence of different pairs fluctuates across different time periods, offering a temporal dimension to our analysis.

import pandas as pd
import plotly.graph_objects as go
import warnings
from tqdm.notebook import tqdm  # To show progress bar

# Suppress the SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
all_crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = all_crypto_counts.nlargest(10).index.tolist()

# Define categories for Y-axis (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# Extract the month from the timestamp
final_df['month'] = final_df['timestamp'].dt.to_period('M')

# Extend the data to include up to 2024-08, even if there are no new data entries
all_months = pd.period_range(final_df['month'].min(), '2024-08', freq='M')

# List of all fiat currencies present in the dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize a list to store all frames
frames = []

# Generate a frame for each month
for month in tqdm(all_months, desc="Generating frames"):
    # Filter the data for the given month
    monthly_data = final_df[final_df['month'] == month].copy()

    # Group the remaining cryptocurrencies into "Others"
    if not monthly_data.empty:
        monthly_data.loc[:, 'crypto_category'] = monthly_data['crypto_currency'].apply(
            lambda x: x if x in top_10_cryptos else 'Others'
        )

    # Initialize an empty matrix with zeros (no pairs)
    matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

    # Fill the matrix with 1 where pairs exist
    if not monthly_data.empty:
        for _, row in monthly_data.iterrows():
            matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

    # Create a frame for this month
    frames.append(go.Frame(
        data=go.Heatmap(
            z=matrix.values,
            x=matrix.columns,
            y=matrix.index,
            colorscale=[[0, '#D3D3D3'], [1, '#4682B4']],  # Light gray for no pair, steel blue for all pairs
            showscale=False
        ),
        name=str(month)
    ))

# Plot using Plotly
fig2 = go.Figure(
    data=frames[0]['data'],  # Use the first month's data as the starting point
    layout=go.Layout(
        title="Top 10 Crypto-Fiat Pair Presence and 'Others' Over Time (All Fiat Currencies)",
        xaxis_title="Fiat Currency",
        yaxis_title="Cryptocurrency",
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(categories))),
            ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
            tickfont=dict(size=10),
            side='left'
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(all_fiats))),
            ticktext=all_fiats  # This is now sorted alphabetically
        ),
        template="plotly_white",
        updatemenus=[dict(type="buttons", showactive=False,
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 500, "redraw": True},
                                                     "fromcurrent": True,
                                                     "transition": {"duration": 300}}]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                       "mode": "immediate",
                                                       "transition": {"duration": 0}}])])],
        sliders=[{
            'currentvalue': {
                'prefix': 'Month: ',
                'font': {'size': 20}
            },
            'pad': {'b': 10},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [{
                'args': [[str(month)], {
                    'frame': {'duration': 500, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 300}
                }],
                'label': str(month),
                'method': 'animate'
            } for month in all_months]
        }]
    ),
    frames=frames  # Add the frames to the figure
)

# Show the plot
fig2.show()


/tmp/ipykernel_33/1497053269.py:26: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



Generating frames:   0%|          | 0/59 [00:00<?, ?it/s]

## 3.2 USD Volume

Block 1: Download prices in USD

In [15]:
pip install coinmetrics-api-client -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [38]:
from coinmetrics.api_client import CoinMetricsClient
import pandas as pd
from tqdm.notebook import tqdm

# Inicializa el cliente de Coin Metrics para la API Comunitaria
client = CoinMetricsClient()

# Define los activos y métricas
assets = ['btc', 'eth', 'usdt']
metrics = ['PriceUSD']

# Define la fecha de inicio (January 1, 2023)
start_time = '2019-01-01'

# Prepara la barra de progreso y descarga los precios
price_data_frames = []
for asset in tqdm(assets, desc="Downloading prices"):
    # Descarga los datos de precios para cada activo
    price_data = client.get_asset_metrics(assets=[asset], metrics=metrics, frequency='1d', start_time=start_time)
    
    # Convierte los datos a un DataFrame de Pandas y agrégalo a la lista
    price_data_frames.append(price_data.to_dataframe())

# Combina todos los DataFrames en uno solo
price_df = pd.concat(price_data_frames)

# Asegúrate de que la columna 'PriceUSD' sea numérica
price_df['PriceUSD'] = pd.to_numeric(price_df['PriceUSD'], errors='coerce')

  asset                      time     PriceUSD
0   btc 2019-01-01 00:00:00+00:00  3808.117832
1   btc 2019-01-02 00:00:00+00:00  3898.197488
2   btc 2019-01-03 00:00:00+00:00  3784.388635
3   btc 2019-01-04 00:00:00+00:00  3827.524594
4   btc 2019-01-05 00:00:00+00:00  3798.613955


Block 2: Processing data and calculating volumeUSD

In [39]:
# Asegúrate de que 'timestamp' en 'final_df' esté en el mismo formato que 'time' en 'price_df'
final_df['timestamp'] = pd.to_datetime(final_df['timestamp']).dt.tz_convert('UTC')

# Filtrar el DataFrame para incluir solo BTC, ETH, y USDT
filtered_df = final_df[final_df['crypto_currency'].isin(['BTC', 'ETH', 'USDT'])]

# Convertir ambos a minúsculas para asegurar coincidencia
filtered_df['book_prefix'] = filtered_df['book_prefix'].str.lower()
price_df['asset'] = price_df['asset'].str.lower()

# Fusionar los DataFrames en función de 'timestamp', 'book_prefix', y 'asset'
merged_df = pd.merge(filtered_df, price_df, left_on=['timestamp', 'book_prefix'], right_on=['time', 'asset'], how='inner')

# Crear la columna 'volumeUSD' realizando la multiplicación del volumen por el precio en USD
merged_df['volumeUSD'] = merged_df['volume'] * merged_df['PriceUSD']

# Crear una nueva base de datos con las columnas relevantes
new_base = merged_df[['timestamp', 'book_prefix', 'volumeUSD', 'fiat_currency']]

# Renombrar las columnas para mayor claridad (opcional)
new_base.columns = ['Date', 'Asset', 'VolumeUSD', 'Country']

                       Date Asset     VolumeUSD Country
0 2023-04-08 00:00:00+00:00   eth   32801.34587     UAH
1 2023-04-09 00:00:00+00:00   eth  71757.279423     UAH
2 2023-04-10 00:00:00+00:00   eth  61124.420792     UAH
3 2023-04-11 00:00:00+00:00   eth   76702.56823     UAH
4 2023-04-12 00:00:00+00:00   eth  67327.813778     UAH


### By country

In [40]:
# Group by month

# Crear una columna de mes
new_base['Month'] = new_base['Date'].dt.to_period('M')

# Agrupar por mes y país y sumar los volúmenes en USD
grouped_data = new_base.groupby(['Month', 'Country'])['VolumeUSD'].sum().unstack().fillna(0)

Country  ARS  AUD  BRL  CZK  EUR  GBP  JPY  MXN              NGN  PLN  RON  \
Month                                                                        
2020-08  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    299198.321926  0.0  0.0   
2020-09  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   7114097.719713  0.0  0.0   
2020-10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   4911690.653923  0.0  0.0   
2020-11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  11259801.262927  0.0  0.0   
2020-12  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  12687626.234301  0.0  0.0   

Country  RUB  TRY  UAH  ZAR  
Month                        
2020-08  0.0  0.0  0.0  0.0  
2020-09  0.0  0.0  0.0  0.0  
2020-10  0.0  0.0  0.0  0.0  
2020-11  0.0  0.0  0.0  0.0  
2020-12  0.0  0.0  0.0  0.0  


/tmp/ipykernel_33/633669558.py:4: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [42]:
# Diccionario de colores basados en las camisetas de fútbol de cada país (dos colores por país)
country_colors = {
    'USD': ['#B22234', '#3C3B6E'],  # Estados Unidos (rojo y azul)
    'EUR': ['#003399', '#FFD700'],  # Unión Europea (azul y amarillo - colores de la bandera)
    'GBP': ['#FFFFFF', '#CC0000'],  # Reino Unido (blanco y rojo)
    'JPY': ['#BC002D', '#FFFFFF'],  # Japón (rojo y blanco)
    'AUD': ['#FFCC00', '#00843D'],  # Australia (amarillo y verde)
    'CAD': ['#FF0000', '#FFFFFF'],  # Canadá (rojo y blanco)
    'CHF': ['#FF0000', '#FFFFFF'],  # Suiza (rojo y blanco)
    'CNY': ['#FFDE00', '#DE2910'],  # China (amarillo y rojo)
    'INR': ['#FF9933', '#138808'],  # India (naranja y verde)
    'RUB': ['#D52B1E', '#0033A0'],  # Rusia (rojo y azul)
    'BRL': ['#F7E03C', '#00A859'],  # Brasil (amarillo y verde)
    'ARS': ['#74ACDF', '#FFFFFF'],  # Argentina (celeste y blanco)
    'NGN': ['#008751', '#FFFFFF'],  # Nigeria (verde y blanco)
    'TRY': ['#E30A17', '#FFFFFF'],  # Turquía (rojo y blanco)
    'MXN': ['#006847', '#FFFFFF'],  # México (verde y blanco)
    'ZAR': ['#006600', '#FFCC00'],  # Sudáfrica (verde y amarillo)
    'PLN': ['#DC143C', '#FFFFFF'],  # Polonia (rojo y blanco)
    'SEK': ['#FFCD00', '#006AA7'],  # Suecia (amarillo y azul)
    'NOK': ['#BA0C2F', '#00205B'],  # Noruega (rojo y azul)
    'DKK': ['#C60C30', '#FFFFFF'],  # Dinamarca (rojo y blanco)
    'HKD': ['#EE1C25', '#FFFFFF'],  # Hong Kong (rojo y blanco)
    'SGD': ['#E00027', '#FFFFFF'],  # Singapur (rojo y blanco)
    'NZD': ['#000000', '#FFFFFF'],  # Nueva Zelanda (negro y blanco)
    'KRW': ['#C60C30', '#003478'],  # Corea del Sur (rojo y azul)
    'THB': ['#2D2A4A', '#A62712'],  # Tailandia (azul y rojo)
    'MYR': ['#010066', '#FFCC00'],  # Malasia (azul y amarillo)
    'IDR': ['#FF0000', '#FFFFFF'],  # Indonesia (rojo y blanco)
    'PHP': ['#0038A8', '#FFFFFF'],  # Filipinas (azul y blanco)
    'VND': ['#DA251D', '#FFFF00'],  # Vietnam (rojo y amarillo)
    'CZK': ['#DA291C', '#003DA5'],  # República Checa (rojo y azul)
    'HUF': ['#C8102E', '#FFFFFF'],  # Hungría (rojo y blanco)
    'RON': ['#FFD700', '#002D62'],  # Rumanía (amarillo y azul)
    'BGN': ['#00966E', '#FFFFFF'],  # Bulgaria (verde y blanco)
    'HRK': ['#FF0000', '#FFFFFF'],  # Croacia (rojo y blanco)
    'RSD': ['#C6363C', '#FFFFFF'],  # Serbia (rojo y blanco)
    'UAH': ['#FFD700', '#0057B7'],  # Ucrania (amarillo y azul)
    'KZT': ['#00AFCA', '#FFD700'],  # Kazajistán (azul y amarillo)
    'AZN': ['#00B140', '#EF3340'],  # Azerbaiyán (verde y rojo)
    'GEL': ['#DD0034', '#FFFFFF'],  # Georgia (rojo y blanco)
    'BYN': ['#D22630', '#FFFFFF'],  # Bielorrusia (rojo y blanco)
    'EGP': ['#CE1126', '#FFFFFF'],  # Egipto (rojo y blanco)
    'ILS': ['#0038A8', '#FFFFFF'],  # Israel (azul y blanco)
    'SAR': ['#00843D', '#FFFFFF'],  # Arabia Saudita (verde y blanco)
    'AED': ['#000000', '#FFFFFF'],  # Emiratos Árabes Unidos (negro y blanco)
    'QAR': ['#8C1D40', '#FFFFFF'],  # Qatar (granate y blanco)
    'KWD': ['#007A3D', '#FFFFFF'],  # Kuwait (verde y blanco)
    'BHD': ['#8C1D40', '#FFFFFF'],  # Baréin (granate y blanco)
    'OMR': ['#D71920', '#FFFFFF'],  # Omán (rojo y blanco)
    'TND': ['#DA291C', '#FFFFFF'],  # Túnez (rojo y blanco)
    'MAD': ['#C1272D', '#4CBB17'],  # Marruecos (rojo y verde)
    'DZD': ['#007A33', '#FFFFFF'],  # Argelia (verde y blanco)
    'TWD': ['#0033A0', '#FFFFFF'],  # Taiwán (azul y blanco)
    'LKR': ['#FEC334', '#B22222'],  # Sri Lanka (amarillo y rojo)
    'PKR': ['#01411C', '#FFFFFF'],  # Pakistán (verde y blanco)
    'BDT': ['#006A4E', '#FFFFFF'],  # Bangladés (verde y blanco)
    'KES': ['#006600', '#FFFFFF'],  # Kenia (verde y blanco)
    'TZS': ['#008C45', '#FFFFFF'],  # Tanzania (verde y blanco)
    'UGX': ['#FFD100', '#CE1126'],  # Uganda (amarillo y rojo)
    'GHS': ['#006B3F', '#FFCC00'],  # Ghana (verde y amarillo)
    'XAF': ['#FFD700', '#00853F'],  # África Central (amarillo y verde)
    'XOF': ['#FFD700', '#00853F'],  # África Occidental (amarillo y verde)
    'XPF': ['#00467F', '#FFFFFF'],  # Polinesia Francesa (azul y blanco)
    'MUR': ['#EC1C24', '#FFD100'],  # Mauricio (rojo y amarillo)
    'SCR': ['#003DA5', '#FFFFFF'],  # Seychelles (azul y blanco)
    'MGA': ['#E20D18', '#007A33'],  # Madagascar (rojo y verde)
    'ZMW': ['#FF9933', '#008751'],  # Zambia (naranja y verde)
    'MWK': ['#CE1126', '#000000'],  # Malaui (rojo y negro)
    'BWP': ['#00ADEF', '#FFFFFF'],  # Botsuana (azul claro y blanco)
    'NAD': ['#00247D', '#FFD100'],  # Namibia (azul y amarillo)
    'SLL': ['#0072C6', '#FFFFFF'],  # Sierra Leona (azul y blanco)
    'LRD': ['#B22234', '#3C3B6E'],  # Liberia (rojo y azul)
    'LSL': ['#003DA5', '#FFFFFF'],  # Lesoto (azul y blanco)
    'SZL': ['#00247D', '#FFD100'],  # Esuatini (azul y amarillo)
    'MZN': ['#C8102E', '#000000'],  # Mozambique (rojo y negro)
    'BIF': ['#00853F', '#FFFFFF'],  # Burundi (verde y blanco)
    'CDF': ['#007FFF', '#FFD700'],  # Congo (azul y amarillo)
    'DJF': ['#00ADEF', '#FFFFFF'],  # Yibuti (azul claro y blanco)
    'ETB': ['#DA291C', '#007A33'],  # Etiopía (rojo y verde)
    'GNF': ['#FFD100', '#CE1126'],  # Guinea (amarillo y rojo)
    'HTG': ['#00205B', '#CE1126'],  # Haití (azul y rojo)
    'NPR': ['#DC143C', '#003153'],  # Nepal (rojo y azul)
    'PGK': ['#D71920', '#000000'],  # Papúa Nueva Guinea (rojo y negro)
    'SHP': ['#00205B', '#FFFFFF'],  # Santa Helena (azul y blanco)
    'SOM': ['#418FDE', '#FFFFFF'],  # Somalia (azul claro y blanco)
    'SSP': ['#CE1126', '#000000'],  # Sudán del Sur (rojo y negro)
    'SYP': ['#CE1126', '#FFFFFF'],  # Siria (rojo y blanco)
    'TTD': ['#E0001B', '#FFFFFF'],  # Trinidad y Tobago (rojo y blanco)
    'TOP': ['#C8102E', '#FFFFFF'],  # Tonga (rojo y blanco)
    'TWD': ['#0033A0', '#FFFFFF'],  # Taiwán (azul y blanco)
    'VUV': ['#FFD100', '#008751'],  # Vanuatu (amarillo y verde)
    'WST': ['#003DA5', '#FFFFFF'],  # Samoa (azul y blanco)
    'XCD': ['#00205B', '#CE1126'],  # Caribe Oriental (azul y rojo)
    'YER': ['#CE1126', '#FFFFFF'],  # Yemen (rojo y blanco)
}

# Asegúrate de que todos los países presentes en el gráfico tengan colores asignados
for country in fiat_currencies:
    if country not in country_colors:
        country_colors[country] = ['#CCCCCC', '#999999']  # Asignar colores grises por defecto si no está definido


In [49]:
import plotly.graph_objects as go

# Crear un gráfico de barras apiladas utilizando dos colores por país
fig = go.Figure()

for idx, country in enumerate(grouped_data.columns):
    # Alternar entre los dos colores del país para las barras
    color = country_colors[country][idx % 2]
    
    fig.add_trace(go.Bar(
        x=grouped_data.index.astype(str),  # Convertir a cadena para que se muestren correctamente los meses
        y=grouped_data[country],
        name=country,
        marker_color=color  # Asignar el color de la camiseta de fútbol
    ))

# Configurar el layout del gráfico
fig.update_layout(
    barmode='stack',  # Apilar las barras
    title='<b>Cryptoasset Exchange Volumes by Local Currencies by Country<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    legend_title_text='Country'
)

# Mostrar el gráfico
fig.show()


### By crypto

In [51]:
# Crear una columna de mes
new_base['Month'] = new_base['Date'].dt.to_period('M')

# Agrupar por mes y criptomoneda y sumar los volúmenes en USD
grouped_data_crypto = new_base.groupby(['Month', 'Asset'])['VolumeUSD'].sum().unstack().fillna(0)

# Verifica los primeros registros para asegurarte de que los datos están listos para el gráfico
print(grouped_data_crypto.head())


Asset    btc              eth  usdt
Month                              
2020-08  0.0    299198.321926   0.0
2020-09  0.0   7114097.719713   0.0
2020-10  0.0   4911690.653923   0.0
2020-11  0.0  11259801.262927   0.0
2020-12  0.0  12687626.234301   0.0


/tmp/ipykernel_33/3700937271.py:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [56]:
import plotly.graph_objects as go

# Definir los colores para cada criptomoneda
crypto_colors = {
    'btc': '#F7931A',   # Naranja para Bitcoin
    'eth': '#627EEA',   # Azul para Ethereum
    'usdt': '#26A17B',  # Verde para Tether
}

# Crear el gráfico de barras apiladas
fig = go.Figure()

for crypto in grouped_data_crypto.columns:
    fig.add_trace(go.Bar(
        x=grouped_data_crypto.index.astype(str),  # Convertir a cadena para que se muestren correctamente los meses
        y=grouped_data_crypto[crypto],
        name=crypto,
        marker_color=crypto_colors[crypto]  # Asignar el color correspondiente a la criptomoneda
    ))

# Configurar el layout del gráfico
fig.update_layout(
    title='<b>Cryptoasset Exchange Volumes by Cryptocurrency<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    barmode='stack',  # Asegura que las barras estén apiladas
    showlegend=True  # Ocultar la leyenda
)

# Mostrar el gráfico
fig.show()


### By type of economy

In [69]:
import pandas as pd

# Supongamos que ya tienes el DataFrame combinado `merged_df` cargado
# Convertir las fechas a formato datetime si aún no está hecho
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

# Clasificación de monedas fiat en AEs y EMDEs
aes_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CZK']  # Economías Avanzadas
emdes_currencies = ['NGN', 'RUB', 'TRY', 'ZAR', 'UAH', 'BRL', 'ERN', 'PLN', 'RON', 'ARS', 'MXN']  # Mercados Emergentes

# Calcular el volumen en dólares (esto ya debería estar hecho en `merged_df`)
merged_df['volumeUSD'] = merged_df['volume'] * merged_df['PriceUSD']

# Seleccionar solo las columnas necesarias
filtered_df = merged_df[['timestamp', 'fiat_currency', 'crypto_currency', 'volumeUSD']]

# Aplicar el filtro de fechas para incluir solo a partir de 2022
filtered_df = filtered_df[filtered_df['timestamp'] >= '2022-01-01']

# Asegurarse de que todas las criptomonedas estén en mayúsculas
filtered_df['crypto_currency'] = filtered_df['crypto_currency'].str.upper()

# Agrupar todas las criptomonedas que no sean BTC o USDT en "Others"
filtered_df['crypto_currency'] = filtered_df['crypto_currency'].apply(lambda x: x if x in ['BTC', 'USDT'] else 'OTHERS')

# Dividir el DataFrame en AEs y EMDEs
aes_df = filtered_df[filtered_df['fiat_currency'].isin(aes_currencies)]
emdes_df = filtered_df[filtered_df['fiat_currency'].isin(emdes_currencies)]

# Agrupar por mes y 'crypto_currency', y sumar 'volumeUSD' para AEs y EMDEs
aes_grouped = aes_df.groupby([pd.Grouper(key='timestamp', freq='ME'), 'crypto_currency']).sum().reset_index()
emdes_grouped = emdes_df.groupby([pd.Grouper(key='timestamp', freq='ME'), 'crypto_currency']).sum().reset_index()


In [70]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear los subplots uno al lado del otro con un tamaño más adecuado
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("AEs", "EMDEs"))

# Definir colores para BTC, USDT y Others
colores = {
    'BTC': '#F7931A',   # Naranja para Bitcoin
    'USDT': '#26A17B',  # Verde para Tether
    'OTHERS': '#B0B0B0'  # Gris para otras criptomonedas
}

# Asegurar el orden de las barras: BTC (primero), USDT (segundo), Others (tercero)
order = ['BTC', 'USDT', 'OTHERS']

# Agregar trazas para AEs
for crypto in order:
    if crypto in aes_grouped['crypto_currency'].unique():
        df_crypto = aes_grouped[aes_grouped['crypto_currency'] == crypto]
        fig.add_trace(go.Bar(
            x=df_crypto['timestamp'],
            y=df_crypto['volumeUSD'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=True
        ), row=1, col=1)

# Agregar trazas para EMDEs
for crypto in order:
    if crypto in emdes_grouped['crypto_currency'].unique():
        df_crypto = emdes_grouped[emdes_grouped['crypto_currency'] == crypto]
        fig.add_trace(go.Bar(
            x=df_crypto['timestamp'],
            y=df_crypto['volumeUSD'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=False  # No mostrar leyenda dos veces
        ), row=1, col=2)

# Personalizar el diseño del gráfico para ajustarlo al ancho de la página
fig.update_layout(
    title='<b>Cryptoasset Exchange Volumes by Cryptocurrency for AEs and EMDEs<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    barmode='stack',
    height=500,  # Ajustar la altura del gráfico para que sea más compacto
    width=800,  # Ajustar el ancho del gráfico para que quepa en la página de Kaggle
    margin=dict(l=20, r=20, t=50, b=20),  # Ajustar los márgenes para maximizar el espacio disponible
    title_font=dict(size=14),  # Reducir el tamaño de la fuente del título para ahorrar espacio
    font=dict(size=10)  # Reducir el tamaño general de la fuente para encajar mejor en la página
)

# Mostrar el gráfico
fig.show()


## Appendix: Automating Graph Exports

This appendix covers the automation of exporting graphs to different platforms. It includes two main subsections: one for publishing graphs on GitHub and another (to be added) for exporting to Plotly Studio Chart.

### Subsection 1: Publishing Graphs on GitHub

This subsection outlines the steps to automate the export of your Plotly graphs to HTML files and publish them to a GitHub repository.

#### Steps:

1. **Export Graphs to HTML**: Save the generated graphs as HTML files.
2. **GitHub Setup**: Configure GitHub credentials and clone the repository.
3. **Move Files to Repository**: Copy the HTML files to the cloned repository.
4. **Commit and Push**: Commit and push the files to GitHub.


In [14]:
import os
import git
from kaggle_secrets import UserSecretsClient

# Retrieve GitHub token securely from Kaggle secrets
user_secrets = UserSecretsClient()
GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")

# Set up the repository URL using the secured token
REPO_URL = "https://{}@github.com/emigiupponi/CryptoFiatVisualizer.git".format(GITHUB_TOKEN)

# Clone the repository if not already done
if not os.path.exists('repo'):
    repo = git.Repo.clone_from(REPO_URL, 'repo')

repo = git.Repo('repo')

# Export graphs to HTML
fig1.write_html("crypto_fiat_animation.html")
fig2.write_html("crypto_fiat_animation_new.html")

# Move HTML files to the repo
os.system('cp crypto_fiat_animation.html repo/')
os.system('cp crypto_fiat_animation_new.html repo/')

# Commit and push to GitHub
repo.index.add(['crypto_fiat_animation.html', 'crypto_fiat_animation_new.html'])
repo.index.commit("Add crypto_fiat animations")
repo.remote(name='origin').push()
